## YOLOX Installation
Some of the installations will take a while, don't worry.

In [ ]:
# check your CUDA version before installing this
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!git clone https://github.com/Megvii-BaseDetection/YOLOX -b 0.3.0

In [ ]:
import os

# set the target folder
yolox_dir = "./YOLOX"
os.chdir(yolox_dir)
print(f"⚠️ Changed working directory to: {os.getcwd()}")

!pip install -U pip && pip install -r requirements.txt
!pip install -v -e .  

In [ ]:
!pip install loguru thop ninja onnx onnxsim onnxruntime onnxruntime-gpu

In [ ]:
!pip install protobuf==3.20.3 setuptools==59.5.0 numpy==1.21.6

In [ ]:
!pip install pycocotools-windows

## Dataset Preparation

#### Train/Validation split data

In [ ]:
import os
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
os.chdir(root_dir)
print(f"⚠️ Changed working directory to: {root_dir}")

In [ ]:
import os

# fill in the path if you want to use your own data
dataset_directory = './02.annotation_data'


# train/validation data path
train_directory = './train'
validation_directory = './validation'


# create training data storage directory
os.makedirs(train_directory, exist_ok=True)
print(f"Training data directory created at: {os.path.abspath(train_directory)}")
# create verification data storage directory
os.makedirs(validation_directory, exist_ok=True)
print(f"Validation data directory created at: {os.path.abspath(validation_directory)}")

In [ ]:
import glob
import shutil
import random
from tqdm import tqdm

# percentage of training data
train_ratio = 0.8

# get copy source file list
annotation_list = sorted(glob.glob(dataset_directory + '/*.xml'))
image_list = sorted(glob.glob(dataset_directory + '/*.jpg'))

file_num = len(annotation_list)

# shuffle
index_list = list(range(file_num - 1))
random.shuffle(index_list)

for count, index in enumerate(tqdm(index_list, desc="Copying dataset")):
    if count < int(file_num * train_ratio):
        # training data
        shutil.copy2(annotation_list[index], train_directory)
        shutil.copy2(image_list[index], train_directory)
    else:
        # validation data
        shutil.copy2(annotation_list[index], validation_directory)
        shutil.copy2(image_list[index], validation_directory)

#### Convert Pascal VOC format to MS COCO format (optional if using custom dataset)

In [ ]:
!git clone https://github.com/Kazuhito00/convert_voc_to_coco.git

In [ ]:
!python convert_voc_to_coco/convert_voc_to_coco.py \
    train train/train_annotations.json \
    --start_image_id=0
!python convert_voc_to_coco/convert_voc_to_coco.py \
    validation validation/validation_annotations.json \
    --start_image_id=10000000

#### Training data directory preparation

In [ ]:
import os
import shutil
from tqdm import tqdm

# Create directories
os.makedirs("dataset/train2017", exist_ok=True)
os.makedirs("dataset/val2017", exist_ok=True)
os.makedirs("dataset/annotations", exist_ok=True)

# Copy JPG images
train_jpgs = [f for f in os.listdir("train") if f.endswith(".jpg")]
for file in tqdm(train_jpgs, desc="Copying training images"):
    shutil.copy2(os.path.join("train", file), "dataset/train2017")

# Copy validation images with progress bar
val_jpgs = [f for f in os.listdir("validation") if f.endswith(".jpg")]
for file in tqdm(val_jpgs, desc="Copying validation images"):
    shutil.copy2(os.path.join("validation", file), "dataset/val2017")

# Copy annotation JSON files
shutil.copy2("train/train_annotations.json", "dataset/annotations")
shutil.copy2("validation/validation_annotations.json", "dataset/annotations")

In [ ]:
import shutil
import os

src = "dataset"
dst = os.path.join("YOLOX", "dataset")

shutil.move(src, dst)
print(f"Moved dataset folder to: {os.path.abspath(dst)}")

## Model Training

#### Copying Configuration
Check and modify configuartion as desired before copying it

![image](https://user-images.githubusercontent.com/37477845/135283504-254ea817-345e-4665-828a-4c6034645ed1.png)


In [ ]:
import shutil
import os

src = os.path.join("03.config", "nano.py")
dst = os.path.join("YOLOX", "nano.py")
shutil.copy2(src, dst)
print(f"Copied nano.py to: {os.path.abspath(dst)}")

src = os.path.join("04.scripts", "demo.py")
dst = os.path.join("YOLOX", "demo.py")
shutil.copy2(src, dst)
print(f"Copied demo.py to: {os.path.abspath(dst)}")

src = os.path.join("04.scripts", "onnx_inference.py")
dst = os.path.join("YOLOX", "onnx_inference.py")
shutil.copy2(src, dst)
print(f"Copied onnx_inference.py to: {os.path.abspath(dst)}")

src = os.path.join("04.scripts", "classes.txt")
dst = os.path.join("YOLOX", "classes.txt")
shutil.copy2(src, dst)
print(f"Copied classes.txt to: {os.path.abspath(dst)}")

#### Training

In [ ]:
import os

# Set the target folder
yolox_dir = "./YOLOX"
os.chdir(yolox_dir)
print(f"⚠️ Changed working directory to: {os.getcwd()}")

In [ ]:
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

url = "https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_nano.pth"
response = requests.get(url, verify=False)
with open("yolox_nano.pth", "wb") as f:
    f.write(response.content)
print("Downloaded successfully!")

In [ ]:
import shutil

shutil.copy2("tools/train.py", "train.py")
print("Copied train.py to current directory.")

In [ ]:
import os
import glob
import subprocess

def setup_complete_vs_environment():
    """
    Complete Visual Studio environment setup for PyTorch C++ extensions
    """
    vs_path = r"C:\Program Files (x86)\Microsoft Visual Studio\2019\BuildTools"
    
    print("Setting up complete Visual Studio environment...")
    
    # 1. Find Windows SDK
    sdk_bases = [
        r"C:\Program Files (x86)\Windows Kits\10",
        r"C:\Program Files\Windows Kits\10"
    ]
    
    include_paths = []
    lib_paths = []
    
    for base in sdk_bases:
        if os.path.exists(base):
            # Find SDK versions
            include_base = os.path.join(base, "Include")
            lib_base = os.path.join(base, "Lib")
            
            if os.path.exists(include_base):
                versions = [d for d in os.listdir(include_base) if d.startswith("10.")]
                if versions:
                    latest_version = max(versions)
                    sdk_include = os.path.join(include_base, latest_version)
                    sdk_lib = os.path.join(lib_base, latest_version)
                    
                    # Add SDK include paths
                    include_paths.extend([
                        os.path.join(sdk_include, "ucrt"),
                        os.path.join(sdk_include, "um"),
                        os.path.join(sdk_include, "shared"),
                        os.path.join(sdk_include, "winrt"),
                        os.path.join(sdk_include, "cppwinrt"),
                    ])
                    
                    # Add SDK library paths
                    lib_paths.extend([
                        os.path.join(sdk_lib, "ucrt", "x64"),
                        os.path.join(sdk_lib, "um", "x64"),
                    ])
                    
                    print(f"Found Windows SDK {latest_version}")
                    break
    
    # 2. Find MSVC tools
    vc_tools_path = os.path.join(vs_path, "VC", "Tools", "MSVC")
    if os.path.exists(vc_tools_path):
        versions = os.listdir(vc_tools_path)
        if versions:
            latest_version = max(versions)
            msvc_base = os.path.join(vc_tools_path, latest_version)
            
            # Add MSVC include paths
            include_paths.extend([
                os.path.join(msvc_base, "include"),
                os.path.join(msvc_base, "atlmfc", "include"),
            ])
            
            # Add MSVC library paths
            lib_paths.extend([
                os.path.join(msvc_base, "lib", "x64"),
                os.path.join(msvc_base, "atlmfc", "lib", "x64"),
            ])
            
            # Add compiler to PATH
            bin_path = os.path.join(msvc_base, "bin", "Hostx64", "x64")
            current_path = os.environ.get('PATH', '')
            os.environ['PATH'] = f"{bin_path};{current_path}"
            
            print(f"Found MSVC {latest_version}")
    
    # 3. Set environment variables
    # Include paths
    current_include = os.environ.get('INCLUDE', '')
    new_include = ';'.join(include_paths + ([current_include] if current_include else []))
    os.environ['INCLUDE'] = new_include
    
    # Library paths
    current_lib = os.environ.get('LIB', '')
    new_lib = ';'.join(lib_paths + ([current_lib] if current_lib else []))
    os.environ['LIB'] = new_lib
    
    # Essential VS environment variables
    os.environ.update({
        'DISTUTILS_USE_SDK': '1',
        'MSSdk': '1',
        'VS160COMNTOOLS': f"{vs_path}\\Common7\\Tools\\",
        'VCINSTALLDIR': f"{vs_path}\\VC\\",
        'WindowsSDKDir': sdk_bases[0] + "\\" if os.path.exists(sdk_bases[0]) else "",
        'PLATFORM': 'x64',
        'PROCESSOR_ARCHITECTURE': 'AMD64',
    })
    
    print(f"Set up {len(include_paths)} include paths")
    print(f"Set up {len(lib_paths)} library paths")
    
    # 4. Verify key libraries exist
    key_libs = ['kernel32.lib', 'msvcprt.lib', 'msvcrt.lib', 'oldnames.lib']
    found_libs = {}
    
    for lib_name in key_libs:
        for lib_path in lib_paths:
            lib_file = os.path.join(lib_path, lib_name)
            if os.path.exists(lib_file):
                found_libs[lib_name] = lib_file
                break
    
    print(f"\nFound libraries: {list(found_libs.keys())}")
    missing_libs = set(key_libs) - set(found_libs.keys())
    if missing_libs:
        print(f"Missing libraries: {list(missing_libs)}")
        
        # Try to find them in other locations
        print("Searching for missing libraries...")
        for lib_name in missing_libs:
            for lib_path in lib_paths:
                if os.path.exists(lib_path):
                    all_libs = [f for f in os.listdir(lib_path) if f.endswith('.lib')]
                    similar = [lib for lib in all_libs if lib_name.split('.')[0] in lib]
                    if similar:
                        print(f"  In {lib_path}: found similar {similar[:3]}")
    
    return len(missing_libs) == 0

def clean_torch_cache():
    """Clean PyTorch extension cache to force recompilation"""
    cache_path = os.path.expanduser("~/.cache/torch_extensions")
    if os.path.exists(cache_path):
        import shutil
        shutil.rmtree(cache_path)
        print("Cleaned PyTorch extensions cache")
    
    # Also clean the specific cache location
    local_cache = r"C:\Users\aarnaizl\AppData\Local\torch_extensions"
    if os.path.exists(local_cache):
        import shutil
        shutil.rmtree(local_cache)
        print("Cleaned local PyTorch extensions cache")

# Run the setup
print("=== Setting up Visual Studio Environment ===")
success = setup_complete_vs_environment()

if success:
    print("\n✅ Environment setup complete!")
    print("🗑️ Cleaning PyTorch cache for fresh compilation...")
    clean_torch_cache()
    print("\n🚀 Ready to run training with fast_cocoeval!")
else:
    print("\n⚠️ Some libraries are missing. You may need to:")
    print("Check Visual Studio Build Tools is installed with the required modules (see README.md)")
    
print("\nYou can now run your training command.")

In [ ]:
import os
import subprocess

command = [
    "python", "train.py",
    "-f", "nano.py",
    "-d", "1",
    "-b", "8",
    "--fp16",
    "-o",
    "-c", "yolox_nano.pth"
]

process = subprocess.Popen(
    command, 
    stdout=subprocess.PIPE, 
    stderr=subprocess.STDOUT, 
    text=True,
    encoding='utf-8',
    errors='replace',
    env=os.environ  # Pass the modified environment
)

for line in process.stdout:
    print(line, end="")

## Inference Test

In [ ]:
import shutil

shutil.copy2("tools/demo.py", "demo.py")
print("Copied demo.py to current directory.")

In [ ]:
import os
import subprocess

TEST_IMAGE_PATH = "../01.image/000050.jpg"
MODEL_PATH = "YOLOX_outputs/nano/best_ckpt.pth"

command = [
    "python", "demo.py", "image",
    "-f", "nano.py",
    "-c", MODEL_PATH,
    "--path", TEST_IMAGE_PATH,
    "--conf", "0.25",
    "--nms", "0.45",
    "--tsize", "640",
    "--save_result",
    "--device", "gpu"
]

process = subprocess.Popen(
    command, 
    stdout=subprocess.PIPE, 
    stderr=subprocess.STDOUT, 
    text=True,
    encoding='utf-8',
    errors='replace',
    env=os.environ  # Pass the modified environment
)

for line in process.stdout:
    print(line, end="")

## Export ONNX Model

In [ ]:
import shutil

shutil.copy2("tools/export_onnx.py", "export_onnx.py")
print("Copied export_onnx.py to current directory.")

In [ ]:
import os
import subprocess

command = [
    "python", "export_onnx.py",
    "--output-name", "yolox_nano.onnx",
    "-n", "yolox-nano",
    "-f", "nano.py",
    "-c", MODEL_PATH
]

process = subprocess.Popen(
    command, 
    stdout=subprocess.PIPE, 
    stderr=subprocess.STDOUT, 
    text=True,
    encoding='utf-8',
    errors='replace',
    env=os.environ  # Pass the modified environment
)

for line in process.stdout:
    print(line, end="")

In [ ]:
# import shutil

# shutil.copy2("demo/ONNXRuntime/onnx_inference.py", "onnx_inference.py")
# print("Copied onnx_inference.py to current directory.")

In [ ]:
import os
import subprocess

command = [
    "python",  "-u", "onnx_inference.py",
    "-m", "yolox_nano.onnx",
    "-i", TEST_IMAGE_PATH,
    "-o", "./",
    "-s", "0.3",
    "--input_shape", "416,416",
]

process = subprocess.Popen(
    command, 
    stdout=subprocess.PIPE, 
    stderr=subprocess.STDOUT, 
    text=True,
    encoding='utf-8',
    errors='replace',
    env=os.environ  # Pass the modified environment
)

for line in process.stdout:
    print(line, end="")


In [ ]:
from PIL import Image

OUTPUT_IMAGE_PATH = "000050.jpg" 
Image.open(OUTPUT_IMAGE_PATH)